In [ ]:
import os
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
#buyers
#name,colour
#ana,red
#bob,green
#carl,red|green

In [ ]:
#sellers
#name,colour
#dave,green
#eden,red|blue
#frank,red

In [ ]:
datadirectory='~/mydata/'
datadirectory=os.path.expanduser(datadirectory)
print(datadirectory)

In [ ]:
sellers_file=datadirectory+'sellers2.csv'
buyers_file=datadirectory+'buyers2.csv'

In [ ]:
print(sellers_file)

In [ ]:
print(buyers_file)

In [ ]:
#Create test files

In [ ]:

def create_test_csv_file(test_file,headers,list_of_customers):
    if os.path.exists(test_file):
        return None

    with open(test_file,'w') as dataoutputfile:
        #Write the csv header
        dataoutputfile.write( str(headers) +"\n")
    
        for seller in list_of_customers:
            line_to_write=str(seller)+"\n"
            dataoutputfile.write(line_to_write)

In [ ]:
headers="name,colour"

list_of_sellers=  [
    
        "dave,green|red|blue",
        "eden,red|blue",
        "frank,red|purple"
    ]

list_of_buyers=  [
    
        "ana,green|red",
        "bob,green",
        "carl,red|green"
    ]

create_test_csv_file(sellers_file,headers,list_of_sellers)
create_test_csv_file(buyers_file,headers,list_of_buyers)



In [ ]:
sellers_df=pd.read_csv(sellers_file,delimiter=',',header=0,quotechar='"')

In [ ]:
buyers_df=pd.read_csv(buyers_file,delimiter=',',header=0,quotechar='"')

In [ ]:
print(sellers_df)

In [ ]:
print(buyers_df)

In [ ]:
#colour is categorical data
#we need to use "one hot encoding"

In [ ]:
sellers_dummy=sellers_df['colour'].str.get_dummies(sep='|')

In [ ]:
buyers_dummy=buyers_df['colour'].str.get_dummies(sep='|')

In [ ]:
print(sellers_dummy)
print(type(sellers_dummy))

In [ ]:
print(buyers_dummy)
print(type(buyers_dummy))

In [ ]:
#!!!!!!WARNING!!!!!!
#THIS SHOULD BE DONE PROGRAMATICALLY
#CHECK THE HEADERS FOR EACH DATAFRAME, FIND THE MISSING ONES, ADD A SERIES OF 0s
#buyers_dummy['blue']=pd.Series([0,0,0])

In [ ]:
def get_dummy_diffs(data_frame_1,data_frame_2):
    
    data_frame_1_values=list(data_frame_1.columns)
    data_frame_2_values=list(data_frame_2.columns)
    
     
    all_values=list(set(data_frame_1_values+data_frame_2_values))
    
    data_frame_1_missing_values_set=set(all_values)-set(data_frame_1_values)
    data_frame_2_missing_values_set=set(all_values)-set(data_frame_2_values)
    
    data_frame_1_missing_values=list(data_frame_1_missing_values_set)
    data_frame_2_missing_values=list(data_frame_2_missing_values_set)
    
    if (data_frame_1_missing_values_set)==set():
        data_frame_1_missing_values=None
        
    if (data_frame_2_missing_values_set)==set():
        data_frame_2_missing_values=None
    
    return all_values,data_frame_1_missing_values,data_frame_2_missing_values

In [ ]:
def pad_dummy(dummy_df,missing_columns_list):
    if missing_columns_list==None:
        return None
    dflength=dummy_df.shape[0]
    
    for column in missing_columns_list:
        dummy_df[column]=pd.Series([0]*dflength)

In [ ]:
all_values,buy_missing,sell_missing =get_dummy_diffs(buyers_dummy,sellers_dummy)

In [ ]:
pad_dummy(buyers_dummy,buy_missing)
pad_dummy(sellers_dummy,sell_missing)

In [ ]:
print(buyers_dummy)

In [ ]:
print(sellers_dummy)

In [ ]:
#axis not in same order

In [ ]:
products=all_values

In [ ]:
buyers_dummy=buyers_dummy.reindex(columns=products)

In [ ]:
sellers_dummy=sellers_dummy.reindex(columns=products)

In [ ]:
print(buyers_dummy)

In [ ]:
print(sellers_dummy)

In [ ]:
cossim=cosine_similarity(sellers_dummy,buyers_dummy)

In [ ]:
print(cossim)

In [ ]:
sellers_names=list(sellers_df['name'])
buyers_names=list(buyers_df['name'])

In [ ]:
print(sellers_names)

In [ ]:
print(buyers_names)

In [ ]:
seller_index=0
buyer_index=0
for seller in sellers_names:
    
    seller_m_index=seller_index%len(sellers_names)
     # We are looping through the sellers matrix in this outer for loop here
     # 0,1,0
     # 1,0,1
     # 1,0,0
    
    for buyer in buyers_names:
        
        #GET THE MATRIX CORDINATES USING THE MOD OPERATOR %  
        #AS EACH TIME YOU COUNT TO THE LENGHTH OF THE MATRIX YOU NEED TO START AT 0 AGAIN
        
        

        buyer_m_index=buyer_index%len(buyers_names)
        # We are looping through the buyers matrix in this inner for loop
        # 1,0,0
        # 0,1,0
        # 1,1,0
        
        similarity=cossim[seller_m_index,buyer_m_index]
        
        print(f"cos_sim( {seller} , {buyer} ) = {similarity}")
        buyer_index=buyer_index+1
    seller_index=seller_index+1
        

In [ ]:
seller_index=0
buyer_index=0
for seller in sellers_names:
    
    print(f"\nCSV Match report for seller: {seller}\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n")
    print("buyer, cosine_similarity,cosine_distance,number_of_products_matched,percent_prods_matched,matched_products")
    for  buyer in buyers_names:
        
        #GET THE MATRIX CORDINATES USING THE MOD OPERATOR %  
        #AS EACH TIME YOU COUNT TO THE LENGHTH OF THE MATRIX YOU NEED TO START AT 0 AGAIN
        seller_m_index=seller_index%len(sellers_names)
        buyer_m_index=buyer_index%len(buyers_names)
        
        #***RETREIVE COSINTE SIMILARITY FROM YOUR PREVIOUS CALCULATION
        cosine_similarity_result=cossim[seller_m_index,buyer_m_index]
        
        #***CALCULATE COSINE DISTANCE
        cosine_distance_result=1-cosine_similarity_result
        #GET THE SELLER AND BUYER VECTORS FROM THE DUMMY MATRICIES
        sellers_vector=list(sellers_dummy.iloc[seller_m_index])
        buyers_vector=list(buyers_dummy.iloc[buyer_m_index])
        
        #***CALCULATE THE NUMBER OF PRODUCTS MATCHING USING DOT PRODUCT
        number_of_products_matched=np.dot(sellers_vector,buyers_vector)
        
        #***CALCULATE THE PERCENTAGE OF PRODUCTS THE SELLER MATCHED TO THE BUYER
        percent_prods_matched=(number_of_products_matched/sum(sellers_vector) ) * 100
        
        
        #***GET A PIPE | DELIMITED LIST OF MATCHED PRODUCTS BETWEEN SELLER AND BUYER
        
        #using list comprehension... as you can not AND two lists, you need to AND the individual list items.
        matched_product_mask=[x and y for x,y in zip(sellers_vector,buyers_vector)]    
        
        #REMIND ME AT SOME POINT TO TELL YOU WHAT ZIP DOES!
        
        matched_product_bools_list=list(map(bool,matched_product_mask))
        matched_product_array=np.array(products)[matched_product_bools_list]
        matched_product_list=list(matched_product_array)
        matched_products="|".join(matched_product_list)
        
        #OUTPUT MATCH RESULTS

        print(f"{buyer}, {cosine_similarity_result},{cosine_distance_result},{number_of_products_matched},{percent_prods_matched},{matched_products}")
        
        
        buyer_index=buyer_index+1
    seller_index=seller_index+1

In [ ]:
seller_index=0
buyer_index=0
for buyer in buyers_names:
    
    print(f"\nCSV Match report for buyer: {buyer}\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n")
    print("seller, cosine_similarity,cosine_distance,number_of_products_matched,percent_prods_matched,matched_products")
    for  seller in sellers_names:
        
        #GET THE MATRIX CORDINATES USING THE MOD OPERATOR %  
        #AS EACH TIME YOU COUNT TO THE LENGHTH OF THE MATRIX YOU NEED TO START AT 0 AGAIN
        seller_m_index=seller_index%len(sellers_names)
        buyer_m_index=buyer_index%len(buyers_names)
        
        #***RETREIVE COSINTE SIMILARITY FROM YOUR PREVIOUS CALCULATION
        cosine_similarity_result=cossim[seller_m_index,buyer_m_index]
        
        #***CALCULATE COSINE DISTANCE
        cosine_distance_result=1-cosine_similarity_result
        
        #GET THE SELLER AND BUYER VECTORS FROM THE DUMMY MATRICIES
        sellers_vector=list(sellers_dummy.iloc[seller_m_index])
        buyers_vector=list(buyers_dummy.iloc[buyer_m_index])
        
        #***CALCULATE THE NUMBER OF PRODUCTS MATCHING USING DOT PRODUCT
        number_of_products_matched=np.dot(sellers_vector,buyers_vector)
        
        #***CALCULATE THE PERCENTAGE OF PRODUCTS THE SELLER MATCHED TO THE BUYER
        percent_prods_matched=(number_of_products_matched/sum(buyers_vector) ) * 100
        
        
        #***GET A PIPE | DELIMITED LIST OF MATCHED PRODUCTS BETWEEN SELLER AND BUYER
        
        #using list comprehension... as you can not AND two lists, you need to AND the individual list items.
        matched_product_mask=[x and y for x,y in zip(sellers_vector,buyers_vector)]    
        
        #REMIND ME AT SOME POINT TO TELL YOU WHAT ZIP DOES!
        
        matched_product_bools_list=list(map(bool,matched_product_mask))
        matched_product_array=np.array(products)[matched_product_bools_list]
        matched_product_list=list(matched_product_array)
        matched_products="|".join(matched_product_list)
        
        #OUTPUT MATCH RESULTS

        print(f"{seller}, {cosine_similarity_result},{cosine_distance_result},{number_of_products_matched},{percent_prods_matched},{matched_products}")
        
        
        seller_index=seller_index+1
    buyer_index=buyer_index+1